In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.dont_write_bytecode
import os
import json
import time
# TOPOLOGIES
from network_topologies.pseudogeant import Pseudogeant
from network_topologies.geant import Geant
from network_topologies.topology import Topology
from network_topologies.topology_preprocessing import preprocess_metadata
# ROUTING ALGORITHMS
from routing_algorithms.dijkstra import dijkstra
from routing_algorithms.dijkstra import calculate_path
from routing_algorithms.dijkstra import set_spt
from routing_algorithms.ear import ear
from routing_algorithms.mora import *
# NETWORK OBJECTS
from utils.network_objects import *
# PA SERVICES
from service_flows.traffic_generator import TrafficGenerator
from itertools import permutations
import random
import numpy as np
from deap import algorithms, base, creator, tools, algorithms
import collections
import matplotlib.pyplot as plt
import import_ipynb

SyntaxError: invalid syntax (mora.py, line 27)

In [49]:
topo = Pseudogeant()
topo.save_topology_info()

# INSTANTIATE TRAFFIC GENERATOR
tg = TrafficGenerator(900)  # 900 s = 15 minuti
# Come faccio a sapere se ci sono nuovi flows?
time.sleep(1)

# Get flows to be routed
service_flows = tg.flows
# get flow from node1 to node2 of type 'premium'


In [54]:
for l in topo.links:
    if l.info["delay"] != 0:
        print(l.info["delay"])

In [ ]:
def optimize_route(topology, flow):
    # number of weights in the tuple -> number of objective functions
    creator.create("FitnessMultiObj", base.Fitness, weights=(-1.0, -1.0, -1.0, -1.0,)) 
    creator.create("Individual", list, fitness=creator.FitnessMultiObj)
    toolbox = base.Toolbox()
    toolbox.register("individual", generate_individual, creator.Individual, flow.starting_node, flow.ending_node, topology)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("mate", crossover_one_point, topology=topology, ind_class=creator.Individual, toolbox=toolbox)
    toolbox.register("select", tools.selNSGA2)
    toolbox.register("mutate", mutate_path, topology=topology, indi_class=creator.Individual)
    
    # FIXME
    toolbox.register("evaluate", evaluate_individual, topology=topology)
    evaluate_SLA = get_evaluate_SLA(flow.SLA, topology)
    penalty = get_penalty(flow.SLA, topology) 
    toolbox.decorate("evaluate", tools.DeltaPenality(evaluate_SLA, 20.0, penalty))

    pop = toolbox.population(n=100)
    #FIXME
    hof = tools.ParetoFront(similar = compare_individuals) #  similar=np.array_equal
    stats = tools.Statistics(lambda ind: ind.fitness.values)

    stats.register("avg", np.mean, axis=0)
    stats.register("std", np.std, axis=0)
    stats.register("min", np.min, axis=0)
    stats.register("max", np.max, axis=0)

    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=10, stats=stats, halloffame=hof)


    return hof

In [59]:
topo.links[0].bandwidth_usage

0.0

In [56]:
service_flows

{'0101premium': {'_id': '0101premium',
  'bandwidth': 1.476,
  'jitter': 1,
  'latency': 1,
  'loss': 1,
  'node1': '01',
  'node2': '01'},
 '0102premium': {'_id': '0102premium',
  'bandwidth': 428.931,
  'jitter': 1,
  'latency': 1,
  'loss': 1,
  'node1': '01',
  'node2': '02'},
 '0103premium': {'_id': '0103premium',
  'bandwidth': 2605.118,
  'jitter': 1,
  'latency': 1,
  'loss': 1,
  'node1': '01',
  'node2': '03'},
 '0104premium': {'_id': '0104premium',
  'bandwidth': 538.453,
  'jitter': 1,
  'latency': 1,
  'loss': 1,
  'node1': '01',
  'node2': '04'},
 '0105premium': {'_id': '0105premium',
  'bandwidth': 6893.432,
  'jitter': 1,
  'latency': 1,
  'loss': 1,
  'node1': '01',
  'node2': '05'},
 '0106premium': {'_id': '0106premium',
  'bandwidth': 276.49,
  'jitter': 1,
  'latency': 1,
  'loss': 1,
  'node1': '01',
  'node2': '06'},
 '0107premium': {'_id': '0107premium',
  'bandwidth': 356.437,
  'jitter': 1,
  'latency': 1,
  'loss': 1,
  'node1': '01',
  'node2': '07'},
 '0108p

In [60]:
for f in service_flows.keys():
    flow_dic = service_flows[f]
    flow_obj = Flow(flow_dic)
    print(flow.)
    #result = optimize_route(topo, flow_obj)
    #path = result[0]
    #topo.apply_service_on_network(flow_dic, path)

[autoreload of routing_algorithms.mora failed: Traceback (most recent call last):
  File "/Users/federicocolangelo/opt/anaconda3/envs/experiments/lib/python3.5/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/federicocolangelo/opt/anaconda3/envs/experiments/lib/python3.5/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Users/federicocolangelo/opt/anaconda3/envs/experiments/lib/python3.5/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/Users/federicocolangelo/opt/anaconda3/envs/experiments/lib/python3.5/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 626, in _exec
  File "<frozen importlib._bootstrap_external>", line 693, in exec_module
  File "<frozen importlib._bootstrap_external>", line 799, in get_code
  File "<frozen importlib._bootstrap_extern

<function compare_individuals at 0x10befe6a8>
gen	nevals	avg             	std                               	min             	max                
0  	100   	[7.95 0.   2.25]	[4.77571984 0.         0.        ]	[3.   0.   2.25]	[21.    0.    2.25]
1  	48    	[7.81 0.   2.25]	[4.6705353 0.        0.       ]   	[3.   0.   2.25]	[21.    0.    2.25]
2  	63    	[7.77 0.   2.25]	[4.62137425 0.         0.        ]	[3.   0.   2.25]	[21.    0.    2.25]
3  	61    	[7.72 0.   2.25]	[4.59147035 0.         0.        ]	[3.   0.   2.25]	[21.    0.    2.25]
4  	55    	[7.63 0.   2.25]	[4.54236722 0.         0.        ]	[3.   0.   2.25]	[21.    0.    2.25]
5  	59    	[7.58 0.   2.25]	[4.51260457 0.         0.        ]	[3.   0.   2.25]	[20.    0.    2.25]


In [39]:
topo.remove_service_from_network(f, path)

In [ ]:
indi = []
for i in range(5000):
    indi.append(generate_individual(list, flow.starting_node, flow.ending_node, topo))

In [ ]:
for ind in indi:
    p = mutate_path(ind, topo, list)
    print(p[0])
    assert not topo.has_loops(p[0])
    assert topo.is_valid(p[0])

In [14]:
(-1.0,) * 3

(-1.0, -1.0, -1.0)